If you haven't yet, start by setting up your environment and datasets by following the instructions in the README. It should be something like:
* `make create_environment`
* `conda activate covid_nlp`
* `make update_environment`
* `make data`

Several common packages that you may want to use (e.g. UMAP, HDBSCAN, enstop, sklearn) have already been added to the `covid_nlp` environment via `environment.yml`. To add more, edit that file and do a:
  ` make update_environment`

In [ ]:
# Quick cell to make jupyter notebook use the full screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Automatically pick up code changes in the `src` module
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import pandas as pd
import numpy as np

In [ ]:
# Useful imports from easydata
from src import paths
from src.data import Dataset
from src import workflow

In [ ]:
from src.data.numba_word_vectorizer import word_word_cooccurence_matrix
from src.data.em_method import em_sparse
from src.utils import RankedPoints

In [ ]:
from src.data.em_method import em_sparse
import en_core_sci_sm # A full spaCy pipeline for biomedical data.
from scispacy.custom_tokenizer import combined_rule_tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from scipy import sparse
from sklearn.preprocessing import normalize
from enstop import PLSA
import umap
import umap.plot
import hdbscan
from wordcloud import WordCloud

In [ ]:
# Some plotting libraries
import matplotlib.pyplot as plt
%matplotlib inline
from bokeh.plotting import show, save, output_notebook, output_file
from bokeh.resources import INLINE
output_notebook(resources=INLINE)

## Load up the dataset

The metadata has been augmented with where the files can be found relative to `paths["interim_data_path"]`

In [ ]:
workflow.available_datasets()

If the previous cell returned an empty list, go back and re-run `make data` as described at the top of this notebook.

In [ ]:
ds_name = 'covid_nlp_20200319'

In [ ]:
# Load the dataset
meta_ds = Dataset.load(ds_name)

In [ ]:
print(meta_ds.DESCR[:457])

In [ ]:
# The processed dataframe is the `data` method of this data source 
meta_df = meta_ds.data
meta_df.head()

## Basics on the dataset

The JSON files given in the `path` column of the metadata dataframe are the papers in `json` format (as dicts)
that include the following keys:
* `paper_id`
* `metadata`
* `abstract`
* `body_text`
* `bib_entries`
* `ref_entries`
* `back_matter`

where the `paper_id` is the sha hash from the medadata.

For example:

In [ ]:
filename = paths['interim_data_path'] / ds_name / meta_df['path'][0]
file = json.load(open(filename, 'rb'))
file.keys()

In [ ]:
abstracts = meta_df.abstract.dropna()

In [ ]:
abstracts[:5]

In [ ]:
len(abstracts)

Shorten abstracts for display

In [ ]:
max_abs_length = 140
short_abstracts = [a[:max_abs_length] for a in abstracts]
meta_df['abstract_length'] = meta_df.abstract.str.len()
data_df = meta_df[meta_df.abstract_length > 0].reset_index()
data_df['short_abstracts'] = short_abstracts

### XXXX Hack around a zero row in the word matrix coming out of word_word_cooccurence_matrix
EM doesn't handle zero rows...

In [ ]:
data_df = data_df[~data_df.abstract.str.contains("subsp")].reset_index()

## Build word and document matrices

In [ ]:
raw_text = data_df.abstract

### Build Word Matrix
As in [05-WordMAP-abstracts.ipynb](05-WordMAP-abstracts.ipynb)

In [ ]:
%%time
raw_word_matrix, token_to_index, index_to_token = word_word_cooccurence_matrix(raw_text, min_df=50)

In [ ]:
# labels of the word matrix
word_array = np.array([index_to_token[x] for x in range(raw_word_matrix.shape[0])])
word_hover_df = pd.DataFrame(word_array, columns=['word'])

In [ ]:
# Without the above hack we get a zero row...
zero_rows = np.where(raw_word_matrix.getnnz(1)==0)[0]
len(zero_rows)

In [ ]:
raw_word_matrix.shape

In [ ]:
%%time
word_matrix_before = TfidfTransformer(norm='l1').fit_transform(raw_word_matrix)
word_matrix_after = TfidfTransformer(norm='l1').fit_transform(raw_word_matrix.T)

In [ ]:
naive_word_matrix = normalize(sparse.hstack([word_matrix_before, word_matrix_after]), norm='l1')

In [ ]:
naive_word_matrix

## Run EM

In [ ]:
background_prior = 5.0

In [ ]:
%%time
word_matrix_before, w_before = em_sparse(word_matrix_before, prior_noise=background_prior)
word_matrix_after, w_after = em_sparse(word_matrix_after, prior_noise=background_prior)

In [ ]:
word_matrix = normalize(sparse.hstack([word_matrix_before, word_matrix_after]), norm='l1')

In [ ]:
word_matrix

## Build Document Matrix
As in [04-DocMAP-abstracts.ipynb](04-WordMAP-abstracts.ipynb), but based on the word vocabulary from above, and using NLTK instead of scispacy.

In [ ]:
%%time
raw_doc_matrix = TfidfVectorizer(vocabulary=token_to_index, norm='l1', min_df=5).fit_transform(raw_text)

In [ ]:
doc_matrix, weights = em_sparse(raw_doc_matrix, prior_noise=background_prior)

## Get word topics

In [ ]:
topic_dimension = 30

In [ ]:
topicer = PLSA(n_components=topic_dimension)

In [ ]:
%%time
topicer.fit(word_matrix)

In [ ]:
word_by_topic = topicer.embedding_

In [ ]:
word_by_topic.shape

## Change Document Matrix to have the same basis

In [ ]:
low_doc_matrix = doc_matrix * word_by_topic

In [ ]:
doc_matrix

In [ ]:
low_doc_matrix.shape

We averaged again by doing a matvec. Do EM again.

In [ ]:
low_doc_matrix, ld_weights = em_sparse(sparse.csr_matrix(low_doc_matrix), prior_noise=background_prior)
low_doc_matrix = low_doc_matrix.todense()

In [ ]:
low_doc_matrix.shape

### Combine to a single word-doc matrix

In [ ]:
word_doc_matrix = np.vstack([word_by_topic, low_doc_matrix])

In [ ]:
word_doc_matrix.shape

## Dimension reduce with UMAP

In [ ]:
%%time
embedding_2d = umap.UMAP(n_components=2, n_neighbors=15,
                         metric='hellinger', init='random',
                         random_state=42).fit(word_doc_matrix)

#### Sort out labels for joint embedding visualization

In [ ]:
from src.utils import get_support_index

In [ ]:
col_indices = [get_support_index(doc_matrix.getrow(i)) for i in range(doc_matrix.shape[0])]
supported_words_array = np.array([" ".join([index_to_token[index_list[i]] for i in range(len(index_list))]) for index_list in col_indices])
doc_hover_df = data_df[['title', 'short_abstracts', 'doi']].copy()
doc_hover_df['word'] = supported_words_array
doc_hover_df['kind'] = ['doc'] * len(doc_hover_df)

In [ ]:
word_hover_df['kind'] = ['word'] * len(word_hover_df)

In [ ]:
hover_df = word_hover_df.merge(doc_hover_df, how="outer")

In [ ]:
p = umap.plot.interactive(embedding_2d, hover_data=hover_df[['word', 'kind', 'title', 'short_abstracts']],
                          labels=hover_df['kind'], width=800, height=800, point_size=1);
show(p)

<img src="../reports/figures/06-topicMAP-abstracts-joint.png" title="Joint embedding visualization" width="800"/>

### Cluster

Naively cluster words and docs to see what comes out for topic words.

XXXX Later cluster docs, take centroids to get topic words.

In [ ]:
min_cluster_size=10

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=min_cluster_size)
clusterer.fit_predict(embedding_2d.embedding_)
labels = clusterer.labels_

In [ ]:
hover_df['cluster'] = labels
value_counts = hover_df.cluster.value_counts()
print(f"Number of clusters: {len(value_counts)}")
print(f"Cluster value counts:\n{value_counts}")

In [ ]:
f = umap.plot.interactive(embedding_2d, labels=hover_df['cluster'],
                          hover_data=hover_df, point_size=1);
show(f)

<img src="../reports/figures/06-topicMAP-abstracts-clusters.png" title="Joint embedding clusters visualization" width="800"/>

In [ ]:
num_no_topic_words = len(hover_df.cluster.value_counts()) - len(hover_df[hover_df.kind=='word'].cluster.value_counts())
num_no_topic_words

In [ ]:
num_no_docs = len(hover_df.cluster.value_counts()) - len(hover_df[hover_df.kind=='doc'].cluster.value_counts())
num_no_docs

By this method, we get 74 document clusters that don't include any word from our vocabulary, and two word clusters with no docs.

In [ ]:
set(hover_df.cluster.value_counts().index).difference(set(hover_df[hover_df.kind=='doc'].cluster.value_counts().index))

### Rank points based on distance to a representative point

Do this naively by ranking words and docs.

In [ ]:
examples = RankedPoints(embedding_2d.embedding_, clusterer, metric='euclidean')

In [ ]:
examples.calculate_all_distances_to_center()
examples.get_all_cluster_rankings()

In [ ]:
hover_df['rank_in_cluster'] = examples.embedding_df['rank_in_cluster']

In [ ]:
hover_df.columns

Get the top words in each cluster (that has words in it)

In [ ]:
num_points = 30
top_cluster_points = {}
top_cluster_points_freq = {}

grouped_by_cluster = hover_df[hover_df.kind=='word'][['word', 'rank_in_cluster', 'cluster']].groupby('cluster')

for cluster_id, group in grouped_by_cluster:
    top_points = group.sort_values('rank_in_cluster', ascending=True).head(num_points)
    inverse_rank = list(range(1, len(top_points)+1))[::-1]
    top_points['inverse_rank'] = inverse_rank
    top_cluster_points_freq[int(cluster_id)] = dict(zip(top_points.word, top_points.inverse_rank))
    top_cluster_points[int(cluster_id)] = '<ol>' + ''.join([f'<li>{r.word}</li>' for _, r in top_points.head(min_cluster_size).iterrows()]) + '</ol>'

Get the top docs in each cluster

In [ ]:
num_points = 50
top_cluster_docs = {}

grouped_by_cluster = hover_df[hover_df.kind=='doc'][['word', 'title', 'doi', 'rank_in_cluster', 'cluster']].groupby('cluster')

for cluster_id, group in grouped_by_cluster:
    top_points = group.sort_values('rank_in_cluster', ascending=True).head(num_points)

    top_cluster_docs[int(cluster_id)] = '<ol>' + ''.join([f'<li><a href="{r.doi}">{r.title}</a></li>' for _, r in top_points.head(min_cluster_size).iterrows()]) + '</ol>'

### Generate word clouds based on ranking

In [ ]:
# generate word cloud for word topic 
def generate_wordcloud(topic_words, topic_num):
    plt.figure(figsize=(16,4))
    plt.axis("off")
    plt.imshow(WordCloud(width=1600, height=400, background_color='black').generate_from_frequencies(topic_words))
    plt.title("Topic " + str(topic_num), loc='left', fontsize=20)

In [ ]:
cluster=201

In [ ]:
HTML(top_cluster_points[cluster])

<ol><li>west</li><li>southern</li><li>migrant</li><li>british</li><li>ksa</li><li>iran</li><li>temperate</li><li>republic</li><li>south</li><li>americas</li></ol>

In [ ]:
generate_wordcloud(top_cluster_points_freq[cluster], cluster)

In [ ]:
HTML(top_cluster_docs[cluster])

<ol><li><a href="http://dx.doi.org/10.3389/fpubh.2018.00385">Point-Of-Care Testing Curriculum and Accreditation for Public Health—Enabling Preparedness, Response, and Higher Standards of Care at Points of Need</a></li><li><a href="10.1101/2020.02.06.20020974">Clinical analysis of 23 cases of 2019 novel coronavirus infection in Xinyang City, Henan Province</a></li><li><a href="http://dx.doi.org/10.1016/j.virol.2009.05.025">Mouse Adenovirus Type 1 Infection of Macrophages</a></li><li><a href="http://dx.doi.org/10.1128/mSphere.00585-18">Gut Virome Analysis of Cameroonians Reveals High Diversity of Enteric Viruses, Including Potential Interspecies Transmitted Viruses</a></li><li><a href="http://dx.doi.org/10.15537/smj.2019.8.24447">Survival of mechanically ventilated patients admitted to intensive care units: Results from a tertiary care center between 2016-2018</a></li><li><a href="http://dx.doi.org/10.1128/JVI.01294-14">Catalytic Function and Substrate Specificity of the Papain-Like Protease Domain of nsp3 from the Middle East Respiratory Syndrome Coronavirus</a></li><li><a href="http://dx.doi.org/10.1126/scitranslmed.aad6873">Host gene expression classifiers diagnose acute respiratory illness etiology</a></li><li><a href="http://dx.doi.org/10.3201/eid2007.140571">MERS Coronavirus in Dromedary Camel Herd, Saudi Arabia</a></li><li><a href="doi.org/10.1101/2020.03.02.20030320">Preliminary estimation of the novel coronavirus disease (COVID-19) cases in Iran: a modelling analysis based on overseas cases and air travel data</a></li><li><a href="http://dx.doi.org/10.3201/eid2007.140296">Detection and Genetic Characterization of Deltacoronavirus in Pigs, Ohio, USA, 2014</a></li></ol>

## View largest topics

In [ ]:
num_topics = 10
top_clusters = value_counts.index[1:num_topics + 1]

In [ ]:
for cluster in top_clusters:
    generate_wordcloud(top_cluster_points_freq[cluster], cluster)